# Master Data Product Lite

> NOTE:  this notebooks likely doesn't work yet

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/mdpLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>


## requirements.txt

In [ ]:
%%capture
!pip install -q rdflib
!pip install -q shapely
!pip install -q pyld
!pip install -q kglab
!pip install -q minio
!pip install -q objdict
!pip install -q shapely
!pip install -q geopandas
!pip install -q oxrdflib

## imports

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import pandas as pd
import geopandas as gpd
from shapely import wkt
# import s3fs
import pyarrow.parquet as pq
import shapely
import requests
import os
import re
import json, io
from pyld import jsonld
import kglab
from minio import Minio
import rdflib
from rdflib import ConjunctiveGraph  #  needed for nquads
from urllib.request import urlopen
from dateutil import parser
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Geod
import oxrdflib # https://github.com/oxigraph/oxrdflib

## Definitions

In [3]:
# pop out last element in a quad to make a triple
def popper(input):
    lines = input.decode().split('\n') # Split input into separate lines
    modified_lines = []

    for line in lines:
        newline = line.replace("http://schema.org", "https://schema.org")
        segments = newline.split(' ')

        if len(segments) > 3:
            segments.pop()   # Remove the last two segment
            segments.pop()
            new_line = ' '.join(segments) + ' .'
            modified_lines.append(new_line)

    result_string = '\n'.join(modified_lines)

    return(result_string)

def contextAlignment(input):
    lines = input.decode().split('\n') # Split input into separate lines
    modified_lines = []

    for line in lines:
        newline = line.replace("http://schema.org", "https://schema.org")

        modified_lines.append(newline)

    result_string = '\n'.join(modified_lines)

    return(result_string)

def publicurls(client, bucket, prefix):
    urls = []
    objects = client.list_objects(bucket, prefix=prefix, recursive=True)
    for obj in objects:
        result = client.stat_object(bucket, obj.object_name)

        if result.size > 0:  #  how to tell if an objet   obj.is_public  ?????
            url = client.presigned_get_object(bucket, obj.object_name)
            # print(f"Public URL for object: {url}")
            urls.append(url)

    return urls

def to_wkt(polygon_string):
    # split the input string into pairs
    pairs = polygon_string.split(',')

    # transform each pair into 'y x' format
    # transformed_pairs = [' '.join(reversed(pair.split())) for pair in pairs]
    transformed_pairs = [' '.join(pair.split()) for pair in pairs]


    # join the transformed pairs with a comma and a space
    transformed_string = ', '.join(transformed_pairs)

    # return the final WKT string
    return f"POLYGON (({transformed_string}))"

def contains_alpha(s):
    if isinstance(s, (int, float)):
      return False
    return any(c.isalpha() for c in s)


## Load Graph(s)

At this point we have the URLs, and we could either loop load all of them or pull one out manually and use.  This section dmonstrates loading and working with one


In [4]:
client = Minio("ossapi.oceaninfohub.org:80",  secure=False) # Create client with anonymous access.
urls = publicurls(client, "public", "graph")
for u in urls:
  print(u)

http://ossapi.oceaninfohub.org/public/graphs/summonedafricaioc_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedaquadocs_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedcioos_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonededmerp_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonededmo_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedemodnet_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinanodc_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinvemardocuments_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinvemarexperts_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinvemarinstitutions_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinvemartraining_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedinvemarvessels_v1_release.nq
http://ossapi.oceaninfohub.org/public/graphs/summonedmarinet

In [17]:
# load single quad graph into a RDFLIB conjunctive graph

u = "http://ossapi.oceaninfohub.org/public/graphs/summonedcioos_v1_release.nq"
# u = "http://ossapi.oceaninfohub.org/public/graphs/summonedoceanexperts_v1_release.nq"

df = urlopen(u)
dg = df.read()
r = contextAlignment(dg)

g = ConjunctiveGraph()
# g = rdflib.ConjunctiveGraph(store="Oxigraph")
g.parse(data=r, format="nquads")
print(len(g))

145779


In [18]:
## Convert the RDFLIB graph to a kglabs graph
namespaces = {
    "sh":   "http://www.w3.org/ns/shacl#" ,
    "schema":   "https://schema.org/" ,
    "geo":      "http://www.opengis.net/ont/geosparql#",
}

kg = kglab.KnowledgeGraph(name = "OIH test", base_uri = "https://oceaninfohub.org/id/", namespaces = namespaces, use_gpus=True, import_graph = g)

## Query Section

In [19]:

# List of URLs
#     "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/baseQuery.rq",
urls = [
    "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/baseQuery.rq",
    "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/sup_geo.rq",
    "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/address_geo.rq",
    "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/sup_temporal.rq",
    "https://raw.githubusercontent.com/iodepo/odis-in/master/SPARQL/searchOIH/dataset.rq"
]

for url in urls:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Extract the file name from the URL and change ".rq" to "rq"
            file_name = url.split("/")[-1].replace(".rq", "rq")
            content = response.text

            # Create a variable with the modified name and store the content
            globals()[file_name] = content
        else:
            print(f"Failed to download URL {url}. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")

In [20]:
print(baseQueryrq)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
prefix prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <https://schema.org/>


SELECT DISTINCT (?s as ?id)  ?type ?name ?url ?description ?headline
WHERE {
         ?s rdf:type ?type .
        OPTIONAL { ?s schema:name ?name . }
        OPTIONAL { ?s schema:description ?description . }
        OPTIONAL { ?s schema:url ?url . }
        OPTIONAL { ?s schema:headline ?headline . }
 }


## Loop on Queries

NOTE, do the queries need isIRI for the subject to avoid resoruces without a top level ID?

In [21]:
qlist = [baseQueryrq, datasetrq,  sup_georq, sup_temporalrq]

# m1 = pd.merge(pdf, geodf, on='id', how='outer')
# mf = pd.DataFrame()
dfl = []
for q in qlist:
  df = kg.query_as_df(q)
  print(len(df))
  if len(df) > 0:
    dfl.append(df)

29970
516025
2538
5550


In [22]:
common_column = ["id", "type"]  # Replace with the actual common column name

# Initialize a merged DataFrame with the first DataFrame
merged_df = dfl[0]

# Iterate through the remaining DataFrames and merge them into the merged_df
for df in dfl[1:]:
    merged_df = pd.merge(merged_df, df, on=common_column, how='inner')

In [23]:
merged_df['id'].nunique()

2015

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670330 entries, 0 to 2670329
Data columns (total 20 columns):
 #   Column                 Dtype 
---  ------                 ----- 
 0   id                     object
 1   type                   object
 2   name_x                 object
 3   url_x                  object
 4   description_x          object
 5   name_y                 object
 6   url_y                  object
 7   description_y          object
 8   license                object
 9   keywords               object
 10  includedInDataCatalog  object
 11  distribution           object
 12  provider               object
 13  publisher              object
 14  creator                object
 15  geotype                object
 16  geompred               object
 17  geom                   object
 18  temporalCoverage       object
 19  datePublished          object
dtypes: object(20)
memory usage: 427.8+ MB


## Post processing

### GeoSpatial

In [25]:
merged_df['filteredgeom'] = merged_df['geom'].apply(lambda x: np.nan if contains_alpha(x) else x)

### Regions
Incorporate Jeff's regions.py which needs

* address (Org, person, Course?
* name (THING, in all)
* spatialFeature (WKT geom column)
* countryOfLastProcessing (vehicle only)

In [26]:
import re

def normalize(s):
    if isinstance(s, str):
      s = s.lower()
      s = re.sub(r"\(.*\)","",s)
      s = re.sub(r"\[.*\]","",s)
      s = re.sub(r"and|the|of","", s)
      s = s.rstrip('.')
      return set(s.split(None))
    else:
      return set(s)

In [29]:
# from . import datashaping
import shapely.geometry
import os, json

import shapely.wkt
import shapely.geometry
from urllib.request import urlopen

with open('./assets/regions-clipped.geojson', 'r') as f:
    geo_regions = json.load(f)['features']
    for r in geo_regions:
        r['shape'] = shapely.geometry.shape(r['geometry'])

# leverage the UNSD API "GeoArea" JSON endpoint, instead of locally-stored CSV
#  see https://unstats.un.org/SDGAPI/swagger/
unsdGeoareaEndpoint = "https://unstats.un.org/SDGAPI/v1/sdg/GeoArea/Tree"
response = urlopen(unsdGeoareaEndpoint)
unsdDataJSON = json.loads(response.read())

# use the "World (total) by continental regions" branch
continentalRegions = unsdDataJSON[1]
continentalRegionsChildren = continentalRegions['children']

# parse the JSON from the API call
countries_dict_with_regions = {}
country_map_list = []

for list_regions in continentalRegionsChildren:
    if list_regions['children'] == None:
        regionName = list_regions['geoAreaName']
        # print('Region name (no children): ' + regionName)
    else:
        regionName = list_regions['geoAreaName']
        # print('Region name: ' + regionName)
        # loop through sub-regions
        for list_subregions in list_regions['children']:
            subRegionName = list_subregions['geoAreaName']
            # print('Sub-region name: ' + subRegionName)
            # loop through intermediate region items
            for list_intermediate_regions in list_subregions['children']:
                if list_intermediate_regions['type'] == 'Region':
                    intermediateRegionName = list_intermediate_regions['geoAreaName']
                    # print('Intermediate region name: ' + intermediateRegionName)
                    # loop through intermediate region children
                    for list_intermediate_region_children in list_intermediate_regions['children']:
                        countryName = list_intermediate_region_children['geoAreaName'].lower()
                        # print('Country name: ' + countryName)
                        countries_dict_with_regions[countryName] = [regionName, subRegionName]
                        country_map_list.append((normalize(countryName), countryName))
                else:
                    countryName = list_intermediate_regions['geoAreaName'].lower()
                    # print('Country name: ' + countryName)
                    countries_dict_with_regions[countryName] = [regionName, subRegionName]
                    country_map_list.append((normalize(countryName), countryName))


def address(address):
    normalized = normalize(address)
    value = list()
    for parts, country in country_map_list:
        if parts <= normalized:
          if country in countries_dict_with_regions:
            value = countries_dict_with_regions[country]
    return value

def name(n):
    normalized = normalize(n)
    value = list()
    for parts, country in country_map_list:
      if parts <= normalized:
        if country in countries_dict_with_regions:
          value = countries_dict_with_regions[country]
    return value


def countryLastProcessing(countryOfLastProcessing):
    normalized = normalize(countryOfLastProcessing)
    value = list()
    for parts, country in country_map_list:
        if parts <= normalized:
          if country in countries_dict_with_regions:
            value = countries_dict_with_regions[country]
    return value


def feature(feature):
    try:
        the_geom = shapely.wkt.loads(feature)
        return [r['properties']['name'] for r in geo_regions if r['shape'].intersects(the_geom)]
    except:
        print("Invalid WKT")
        return list()

In [30]:
import pandas as pd

data1 = ["Marine Science Country Profiles : Kenya",
        "The fisheries of Barbados and some of their problems",
        "Fiji : Where's the data?", 'POLYGON ((-95.5 19.5,-95.5 31.5,-73.5 31.5,-73.5 19.5,-95.5 19.5))']

data2 = ["Marine Science Country Profiles : Kenya",
        "The fisheries of Barbados and some of their problems",
        "Fiji : Where's the data?"]

dftest = pd.DataFrame({'name': data1})
dftest.head()

,name
0,Marine Science Country Profiles : Kenya
1,The fisheries of Barbados and some of their pr...
2,Fiji : Where's the data?
3,"POLYGON ((-95.5 19.5,-95.5 31.5,-73.5 31.5,-73..."


In [31]:
# merged_df['region'] = merged_df['name'].apply(lambda x: x + [item for item in name(x) if item not in x] if x else x)
dftest['nregion'] = dftest['name'].apply(lambda x: name(x)  if x else x)
dftest['aregion'] = dftest['name'].apply(lambda x: address(x)  if x else x)
dftest['cregion'] = dftest['name'].apply(lambda x: countryLastProcessing(x)  if x else x)
dftest['fregion'] = dftest['name'].apply(lambda x: feature(x)  if x else x)

dftest.head()

Invalid WKT
Invalid WKT
Invalid WKT


,name,nregion,aregion,cregion,fregion
0,Marine Science Country Profiles : Kenya,"[Africa, Sub-Saharan Africa]","[Africa, Sub-Saharan Africa]","[Africa, Sub-Saharan Africa]",[]
1,The fisheries of Barbados and some of their pr...,"[Americas, Latin America and the Caribbean]","[Americas, Latin America and the Caribbean]","[Americas, Latin America and the Caribbean]",[]
2,Fiji : Where's the data?,"[Oceania, Melanesia]","[Oceania, Melanesia]","[Oceania, Melanesia]",[]
3,"POLYGON ((-95.5 19.5,-95.5 31.5,-73.5 31.5,-73...",[],[],[],"[Caribbean Sea, Atlantic Ocean, Latin America ..."


In [32]:
def g(df):
    df['region'] = df[['nregion', 'aregion','cregion', 'fregion']].apply(lambda x: list(set(x[0] + x[1] + x[2]+ x[3])), axis=1)
    del df['nregion']
    del df['aregion']
    del df['cregion']
    del df['fregion']
    return df

In [33]:
dftest = g(dftest.copy())

dftest.head()

,name,region
0,Marine Science Country Profiles : Kenya,"[Africa, Sub-Saharan Africa]"
1,The fisheries of Barbados and some of their pr...,"[Americas, Latin America and the Caribbean]"
2,Fiji : Where's the data?,"[Melanesia, Oceania]"
3,"POLYGON ((-95.5 19.5,-95.5 31.5,-73.5 31.5,-73...","[Atlantic Ocean, Caribbean Sea, Latin America ..."


In [34]:
def make_pairs(ll):
  """Makes pairs of coordinates from a list of coordinates.

  Args:
    ll: A list of coordinates.

  Returns:
    A list of pairs of coordinates.
  """

  coords = []
  for i in range(0, len(ll), 2):
    coords.append((ll[i], ll[i+1]))

  return coords

In [35]:
def gj(geom, value):
  test = geom.split()
  test = [float(x) for x in test]
  if len(test) < 2:
    return None

  cp = make_pairs(test)

  if len(cp) == 1:
    # print("POINT")
    geom = shapely.Point(cp)
  elif len(cp) == 2:
    # print("BOX")
    geom = shapely.box(cp[0][0], cp[0][1], cp[1][0], cp[1][1])
  else:
    # print("POLYGON")
    geom = shapely.Polygon(cp)

  if value == "centroid":
    return geom.centroid
  elif value == "length":
    return geom.length
  elif value == "area":
    geod = Geod(ellps="WGS84")
    area = abs(geod.geometry_area_perimeter(geom)[0])
    return area
  elif value == "wkt":
    return shapely.to_wkt(geom)
  elif value == "geojson":
    return shapely.to_geojson(geom)
  else:
    return None

In [36]:
# merged_df['dt_endDate'] = merged_df['temporalCoverage'].apply(lambda x: re.split("/", x)[1] if "/" in x else np.nan)

merged_df['centroid'] = merged_df['filteredgeom'].apply(lambda x: gj(str(x), "centroid"))
merged_df['length'] = merged_df['filteredgeom'].apply(lambda x: gj(str(x), "length"))
merged_df['area'] = merged_df['filteredgeom'].apply(lambda x: gj(str(x), "area"))
merged_df['wkt'] = merged_df['filteredgeom'].apply(lambda x: gj(str(x), "wkt"))
merged_df['geojson'] = merged_df['filteredgeom'].apply(lambda x: gj(str(x), "geojson"))

In [37]:
# merged_df[].head()
print(merged_df["wkt"][1])

POLYGON ((47.974647 -129.08715, 47.974647 -129.08715, 47.974647 -129.08715, 47.974647 -129.08715, 47.974647 -129.08715))


### Temporal

In [38]:
merged_df['temporalCoverage'] = merged_df['temporalCoverage'].astype('str')  # fine to make str since we don't use in the solr JSON
merged_df['dt_startDate'] = merged_df['temporalCoverage'].apply(lambda x: re.split("/", x)[0] if "/" in x else np.nan)
merged_df['dt_endDate'] = merged_df['temporalCoverage'].apply(lambda x: re.split("/", x)[1] if "/" in x else np.nan)
merged_df['n_startYear'] = merged_df['dt_startDate'].apply(lambda x: parser.parse(x).year if "-" in str(x) else np.nan)
merged_df['n_endYear'] = merged_df['dt_endDate'].apply(lambda x: parser.parse(x).year if "-" in str(x) else np.nan)

In [39]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670330 entries, 0 to 2670329
Data columns (total 30 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   type                   object 
 2   name_x                 object 
 3   url_x                  object 
 4   description_x          object 
 5   name_y                 object 
 6   url_y                  object 
 7   description_y          object 
 8   license                object 
 9   keywords               object 
 10  includedInDataCatalog  object 
 11  distribution           object 
 12  provider               object 
 13  publisher              object 
 14  creator                object 
 15  geotype                object 
 16  geompred               object 
 17  geom                   object 
 18  temporalCoverage       object 
 19  datePublished          object 
 20  filteredgeom           object 
 21  centroid               object 
 22  length            

In [40]:
merged_df.head()

,id,type,name_x,url_x,description_x,name_y,url_y,description_y,license,keywords,...,filteredgeom,centroid,length,area,wkt,geojson,dt_startDate,dt_endDate,n_startYear,n_endYear
0,<https://catalogue.cioos.ca/dataset/2d2f65b5-c...,schema:Dataset,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,None,Oceans,...,47.974646666 -129.08715 47.974646666 -129.08715,POINT (47.974646666 -129.08715),0.0,NaN,"POLYGON ((47.974647 -129.08715, 47.974647 -129...","{""type"":""Polygon"",""coordinates"":[[[47.97464666...",2018-06-28,2021-08-24,2018.0,2021.0
1,<https://catalogue.cioos.ca/dataset/2d2f65b5-c...,schema:Dataset,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,None,Oceans,...,47.974646666 -129.08715 47.974646666 -129.08715,POINT (47.974646666 -129.08715),0.0,NaN,"POLYGON ((47.974647 -129.08715, 47.974647 -129...","{""type"":""Polygon"",""coordinates"":[[[47.97464666...",2018-06-28,2021-08-24,2018.0,2021.0
2,<https://catalogue.cioos.ca/dataset/2d2f65b5-c...,schema:Dataset,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,None,Oceans,...,47.974646666 -129.08715 47.974646666 -129.08715,POINT (47.974646666 -129.08715),0.0,NaN,"POLYGON ((47.974647 -129.08715, 47.974647 -129...","{""type"":""Polygon"",""coordinates"":[[[47.97464666...",2018-06-28,2021-08-24,2018.0,2021.0
3,<https://catalogue.cioos.ca/dataset/2d2f65b5-c...,schema:Dataset,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,None,Oceans,...,47.974646666 -129.08715 47.974646666 -129.08715,POINT (47.974646666 -129.08715),0.0,NaN,"POLYGON ((47.974647 -129.08715, 47.974647 -129...","{""type"":""Polygon"",""coordinates"":[[[47.97464666...",2018-06-28,2021-08-24,2018.0,2021.0
4,<https://catalogue.cioos.ca/dataset/2d2f65b5-c...,schema:Dataset,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,Endeavour North Conductivité/Température/Profo...,https://catalogue.cioos.ca/dataset/2d2f65b5-ca...,Ce Sea-Bird Microcat SBE37SIP 5695 a été déplo...,None,Oceans,...,47.974646666 -129.08715 47.974646666 -129.08715,POINT (47.974646666 -129.08715),0.0,NaN,"POLYGON ((47.974647 -129.08715, 47.974647 -129...","{""type"":""Polygon"",""coordinates"":[[[47.97464666...",2018-06-28,2021-08-24,2018.0,2021.0


In [43]:
# transforms needed for aggregation
merged_df['keywords'] = merged_df['keywords'].astype(str)  #  why is this needed?


if "geom" in merged_df.columns:
    merged_df['geom'] = merged_df['geom'].astype(str)  # why is this needed?
    merged_df['filteredgeom'] = merged_df['filteredgeom'].astype(str)  # why is this needed?
    merged_df['centroid'] = merged_df['centroid'].astype(str)  # why is this needed?

In [45]:
agg_dict = {'keywords': ', '.join,
            'type': 'first',
            'name': ', '.join,
            'description': ', '.join,
            'url': ', '.join,
            'geotype': 'first',
            'geompred': 'first',
            'geom': 'first',
            'temporalCoverage': 'first',
            'datePublished': 'first',
            'license': 'first',
            'creator': 'first',
            'includedInDataCatalog': 'first',
            'distribution': 'first',
            'publisher': 'first',
            'filteredgeom': 'first',
            'region': 'first',
            'dt_startDate': 'first',
            'dt_endDate': 'first',
            'n_startYear': 'first',
            'n_endYear': 'first',
            'centroid': 'first',
            'length': 'first',
            'area': 'first',
            'wkt': 'first',
            'geojson': 'first'}

for col in agg_dict.copy():
    if col not in merged_df.columns:
        del agg_dict[col]

In [47]:
mf = merged_df.groupby('id').agg(agg_dict).reset_index()


In [48]:
mf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2015 entries, 0 to 2014
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     2015 non-null   object 
 1   keywords               2015 non-null   object 
 2   type                   2015 non-null   object 
 3   geotype                2014 non-null   object 
 4   geompred               2014 non-null   object 
 5   geom                   2015 non-null   object 
 6   temporalCoverage       2015 non-null   object 
 7   datePublished          2015 non-null   object 
 8   license                2015 non-null   object 
 9   creator                2009 non-null   object 
 10  includedInDataCatalog  2015 non-null   object 
 11  distribution           2013 non-null   object 
 12  publisher              2015 non-null   object 
 13  filteredgeom           2015 non-null   object 
 14  dt_startDate           1813 non-null   object 
 15  dt_e

In [50]:
mf.to_parquet('mdpProduct.parquet')
# mf.to_csv('mdpProduct.csv')